<a href="https://colab.research.google.com/github/talhaanwarch/Sarcasm-nlp/blob/master/multi_input_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install tensorflow==2.1.0
!pip install sentencepiece
!pip install tensorflow-hub
#!wget --quiet https://raw.githubusercontent.com/google-research/ALBERT/master/tokenization.py

In [3]:
cd /content/drive/My\ Drive/dataset/sarcasm

/content/drive/My Drive/dataset/sarcasm


In [4]:
ls

__pycache__/
sarcasm_detection_shared_task_reddit_testing.jsonl
sarcasm_detection_shared_task_reddit_training.jsonl
sarcasm_detection_shared_task_twitter_testing.jsonl
sarcasm_detection_shared_task_twitter_training.jsonl
tokenization.py


In [0]:
import pandas as pd
import numpy as np

In [0]:
twitter_train=pd.read_json('sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)

In [7]:
twitter_train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [0]:
twitter_train['response']=twitter_train['response'].str.replace('@USER', "") 
twitter_train['response']=twitter_train['response'].str.replace('\d+', '')
twitter_train['response']=twitter_train['response'].str.lower()
twitter_train['response']=twitter_train['response'].str.replace('[^\w\s]','')


In [9]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,[A minor child deserves privacy and should be ...
1,SARCASM,trying to protest about talking about him a...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,he makes an insane about of money from the ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,meanwhile trump wont even release his sat sc...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,[Man ... y ’ all gone “ both sides ” the apoca...


In [0]:
twitter_train['context']=twitter_train['context'].apply(lambda x: ','.join(map(str, x)))
twitter_train['context']=twitter_train['context'].str.replace('@USER', "") 
twitter_train['context']=twitter_train['context'].str.lower()
twitter_train['context']=twitter_train['context'].str.replace('[^\w\s]','')
twitter_train['context']=twitter_train['context'].str.replace('\d+', '')


In [11]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [12]:
len(twitter_train)

5000

In [13]:
from collections import Counter
Counter(twitter_train['label'])

Counter({'NOT_SARCASM': 2500, 'SARCASM': 2500})

In [0]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_label=le.fit_transform(twitter_train['label'])

#BERT

In [0]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
train_response = bert_encode(twitter_train.response, tokenizer, max_len=160)
train_context = bert_encode(twitter_train.context, tokenizer, max_len=160)
train_labels=twitter_train.label.values
#test_input = bert_encode(test_bert.values, tokenizer, max_len=160)

In [19]:
train_labels[0]

'SARCASM'

In [20]:
twitter_train.response[0:5]

0       i dont get this  obviously you do care or y...
1      trying to protest about  talking about him a...
2       he makes an insane about of money from the ...
3      meanwhile trump wont even release his sat sc...
4      pretty sure the antilincoln crowd claimed th...
Name: response, dtype: object

In [0]:
 max_len=160

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,concatenate,Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [0]:
contex_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_word_ids")
contex_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_mask")
contex_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_segment_ids")
_, context_sequence_output = bert_layer([contex_input_word_ids, contex_input_mask, contex_segment_ids])
context_clf_output = context_sequence_output[:, 0, :]


reponse_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_word_ids")
reponse_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_mask")
reponse_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_segment_ids")
_, reponset_sequence_output = bert_layer([reponse_input_word_ids, reponse_input_mask, reponse_segment_ids])
reponset_clf_output = reponset_sequence_output[:, 0, :]

conc=concatenate([context_clf_output,reponset_clf_output])
out = Dense(1, activation='sigmoid')(conc)
    
model = Model(inputs=[contex_input_word_ids, contex_input_mask, contex_segment_ids,reponse_input_word_ids,reponse_input_mask,reponse_segment_ids], outputs=out)
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    


In [0]:
train_gen=[train_context[0],train_context[1],train_context[2],train_response[0],train_response[1],train_response[2]]

In [0]:

train_history = model.fit(
    train_gen, train_label,
    validation_split=0.2,
    epochs=5,
    batch_size=8)

Train on 4000 samples, validate on 1000 samples
Epoch 1/5


#test

In [0]:
twitter_test=pd.read_json('sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
twitter_test.head()

In [0]:
twitter_test['response']=twitter_test['response'].str.replace('@USER', "") 
twitter_test['response']=twitter_test['response'].str.replace('\d+', '')
twitter_test['response']=twitter_test['response'].str.lower()
twitter_test['response']=twitter_test['response'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].apply(lambda x: ','.join(map(str, x)))
twitter_test['context']=twitter_test['context'].str.replace('@USER', "") 
twitter_test['context']=twitter_test['context'].str.lower()
twitter_test['context']=twitter_test['context'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].str.replace('\d+', '')


In [0]:
twitter_test.head()

In [0]:
test_response = bert_encode(twitter_test.response, tokenizer, max_len=160)
test_context = bert_encode(twitter_test.context, tokenizer, max_len=160)

In [0]:
test_gen=[test_context[0],test_context[1],test_context[2],test_response[0],test_response[1],test_response[2]]

In [0]:
y_pred=model.predict(test_gen)

In [0]:
y_pred=y_pred.round()

In [0]:
y_pred[0:5]

In [0]:
y_pred=le.inverse_transform(y_pred.ravel().astype('int16'))

In [0]:
y_test=pd.DataFrame(y_pred,columns=['label'])

In [0]:
y_test=pd.concat([twitter_test['id'], y_test], axis=1)

In [0]:
y_test.head()

In [0]:
y_test.to_csv('/content/answer.txt',sep=',',index=False,header=None)
